# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model_temp import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader = DataLoader(trainset, batch_size=20, collate_fn=collate_fn, shuffle=True)

landmarks, len_landmark, label, len_label = next(iter(trainloader))


In [4]:
vocabulary = create_vocabulary(blank="@")

In [ ]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}

In [23]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]

label_t = char_to_index_batch(label, vocabulary)

#target_indices = [char_to_index[char] for char in labels]
#target_tensor = torch.tensor(target_indices)

In [6]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)



In [ ]:
landmarks.shape

In [ ]:
print(torch.reshape(landmarks[0], (landmarks[0].shape[0], landmarks[0].shape[1]*landmarks[0].shape[2]))[None,:,:].shape)
print(len_landmark[0][None, None].shape)

In [ ]:
landmarks.shape

In [24]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    
    for i in range(landmarks.shape[0]):
        landmarks[i] = landmarks[i].to(device) 
        label_t[i] = label_t[i].to(device)
        optimizer.zero_grad()
        target_tensor = label_t[i]
        reshaped_landmark = torch.reshape(landmarks[i], (landmarks[i].shape[0], landmarks[i].shape[1]*landmarks[i].shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark[i] = len_landmark[i].to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



Epoch [10/10000], Loss: 1.8985503911972046
Epoch [20/10000], Loss: 1.6644002199172974
Epoch [30/10000], Loss: 1.5414624214172363
Epoch [40/10000], Loss: 1.3623918294906616
Epoch [50/10000], Loss: 1.2469836473464966
Epoch [60/10000], Loss: 1.1365450620651245
Epoch [70/10000], Loss: 1.0001591444015503


KeyboardInterrupt: 

In [28]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########

#trainset = vocadataset("train", landmark=True)
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader_ = DataLoader(trainset_, batch_size=1, collate_fn=collate_fn, num_workers=8)

###########à

In [29]:



# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    for i, (landmarks, len_landmark, label, len_label) in enumerate(trainloader):

        # reshape the batch from [batch_size, frame_size, num_landmark, 3] to [batch_size, frame_size, num_landmark * 3] 
        landmarks = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1], landmarks.shape[2]*landmarks.shape[3]))
        
        #variable to recover later the target sequences
        label_list = label

        # label char to index
        label = char_to_index_batch(label, vocabulary)

        # move data to GPU!
        landmarks = landmarks.to(device)
        len_landmark = len_landmark.to(device)
        label = label.to(device)
        len_label = len_label.to(device)
        optimizer.zero_grad()

        output = model(landmarks,len_landmark )
        output = output.permute(1, 0, 2)#had to permute for the ctc loss. it acceprs [seq_len, batch_size, "num_class"]

        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), label, len_landmark, len_label)
        loss.backward()
        optimizer.step()

        """e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 1 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() """
        print(i)
        if(i == 10):
            break
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5


KeyboardInterrupt: 

In [11]:
#torch.save(model.state_dict(), "models/model20sent_2.pt")

In [ ]:
# Decode the output sequence
output_indices = torch.argmax(output, dim=2).squeeze(1)
output_sequence = ''.join([vocabulary[index] for index in output_indices])

print("Target Sequence:", labels.replace("@","").replace("#",""))
print("Decoded Output:", process_string(output_sequence))